# Baselines

## Import

In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas
from tqdm import tqdm

In [ ]:
import pandas as pd
import glob

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## Load Data

In [ ]:
italy_segmented_files = glob.glob("dataset_anemia/Italy/*/*forniceal_palpebral.png")
india_segmented_files = glob.glob("dataset_anemia/India/*/*forniceal_palpebral.png")
segmented_files = india_segmented_files + italy_segmented_files

In [ ]:
print(india_segmented_files[0])

In [ ]:
# y = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_base_italy.npy") #/30
# X = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_base_italy.npy")/255
yit = np.load(r"y_base_italy.npy")
# x = np.load(r"/content/X_new.npy")/255
x = np.load(r"X_new_RGB.npy")

In [ ]:
yin = np.load(r"y_india.npy")

In [ ]:
yin.shape

In [ ]:
y = np.concatenate((yit , yin))
y.shape

In [ ]:
x.shape

## Augmentation Functions

In [ ]:
Folder_name = "dataset_anemia/Seg_task_data"
Extension=".jpg"

def scale_image(image,name):
    image = cv2.resize(image,(480,640), interpolation = cv2.INTER_CUBIC)
    l = name.split("/")
    #cv2.imwrite(Folder_name+"/Scale/"+'india_'+l[2]+Extension, image)
    return image

def scale_image_or(image,name):
    image = cv2.resize(image,(480,640), interpolation = cv2.INTER_CUBIC)
    l = name.split("/")
    #cv2.imwrite(Folder_name+"/Scale/"+'india_'+l[2]+Extension, image)
    return image
def scale_image_seg(image,name):
    image = cv2.resize(image,(480,640), interpolation = cv2.INTER_CUBIC)
    l = name.split("/")
    #cv2.imwrite(Folder_name+"/Scale/"+'india_'+l[2]+Extension, image)
    return image

def translation_image(image,x,y,name):
    rows, cols ,c= image.shape
    M = np.float32([[1, 0, x], [0, 1, y]])
    image = cv2.warpAffine(image, M, (cols, rows))
    l = name.split("/")
    return image
    #cv2.imwrite(Folder_name + "/Translation/"+'india_'+l[2]+"_"+ str(x) + str(y) + Extension, image)

def rotate_image(image,deg,name):
    rows, cols,c = image.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), deg, 1)
    image = cv2.warpAffine(image, M, (cols, rows))
    l = name.split("/")
    return image
    #cv2.imwrite(Folder_name + "/Rotate/"+'india_'+l[2]+"_"+str(deg) + Extension, image)

def flip_image(image,name):
    imageh = cv2.flip(image, 0)
    imagev = cv2.flip(image, 1)
    l = name.split("/")
    return imageh,imagev
    #cv2.imwrite(Folder_name + "/Flip/"+'india_'+l[2]+ "_"+str(0)+Extension, imageh)
    #cv2.imwrite(Folder_name + "/Flip/"+'india_'+l[2]+ "_"+str(1)+Extension, imagev)

## Data Augmentation

In [ ]:
x_aug_it = []
y_aug_it = []
for a in range(x.shape[0]):
    i = x[a]
    filename = 'a'
    x_aug_it.append(i)
    y_aug_it.append(yit[a])

    x_aug_it.append(translation_image(i,50,50,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(translation_image(i,-50,50,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(translation_image(i,50,-50,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(translation_image(i,-50,-50,filename))
    y_aug_it.append(yit[a])

    x_aug_it.append(rotate_image(i,45,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,90,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,135,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,180,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,225,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,270,filename))
    y_aug_it.append(yit[a])
    x_aug_it.append(rotate_image(i,315,filename))
    y_aug_it.append(yit[a])

    x_aug_it.append(flip_image(i,filename)[0])
    y_aug_it.append(yit[a])
    x_aug_it.append(flip_image(i,filename)[1])
    y_aug_it.append(yit[a])

In [ ]:
x_aug_it = np.array(x_aug_it)
x_aug_it.shape

In [ ]:
y_aug_it = np.array(y_aug_it)
y_aug_it.shape

In [ ]:
x_aug_it = x_aug_it/255
y_aug_it = y_aug_it/20

In [ ]:
#for U net
for i in india_segmented_files:
    im = cv2.imread(i)
    
    l = i.split("/")
    filename = i
    
    image=scale_image(im,filename)
    path_folder = "/".join(l[:3])+"/*.jpg"
    file = glob.glob(path_folder)

    orig_im=cv2.imread(file[0])
    plt.imshow(im)
    plt.imshow(orig_im)
    exit()

In [ ]:
#augmentation
for i in india_segmented_files:
    im = cv2.imread(i)
    xc, yc = np.nonzero(im[:,:,0])
    xl,xr = xc.min(),xc.max()
    yl,yr = yc.min(),yc.max()
    im = im[xl:xr+1, yl:yr+1,:]
    
    l = i.split("/")
    filename = i
    
    image=scale_image(im,filename)

    translation_image(image,50,50,filename)
    translation_image(image,-50,50,filename)
    translation_image(image,50,-50,filename)
    translation_image(image,-50,-50,filename)

    rotate_image(image,45,filename)
    rotate_image(image,90,filename)
    rotate_image(image,135,filename)
    rotate_image(image,180,filename)
    rotate_image(image,225,filename)
    rotate_image(image,270,filename)
    rotate_image(image,315,filename)

    flip_image(image,filename)

In [ ]:
x_seg=[]
y_seg=[]
for i in italy_segmented_files:
    im = cv2.imread(i)
    xc, yc = np.nonzero(im[:,:,0])
    xl,xr = xc.min(),xc.max()
    yl,yr = yc.min(),yc.max()
    im = im[xl:xr+1, yl:yr+1,:]
    x_seg.append(cv2.resize(im,(480,640)))
    l = i.split("/")
    y_seg.append(yit[int(l[2])-1])

for i in india_segmented_files:
    im = cv2.imread(i)
    xc, yc = np.nonzero(im[:,:,0])
    xl,xr = xc.min(),xc.max()
    yl,yr = yc.min(),yc.max()
    im = im[xl:xr+1, yl:yr+1,:]
    x_seg.append(cv2.resize(im,(480,640)))
    l = i.split("/")
    y_seg.append(yin[int(l[2])-1])

In [ ]:
x_seg = np.array(x_seg)
y_seg = np.array(y_seg)

In [ ]:
x_seg.shape

In [ ]:
y_seg.shape

In [ ]:
plt.imshow(x_seg[1][:,:,:])

In [ ]:
data = pd.read_excel("dataset_anemia/Italy/Italy.xlsx")

In [ ]:
data['Hgb'].isna

In [ ]:
Age = data['Age'].astype(np.float32)
Hgb = data['Hgb'].astype(np.float32)
plt.scatter(Age,Hgb)
plt.xlabel('Age')
plt.ylabel('Hgb')

In [ ]:
gender = data['Gender']
Hgb = data['Hgb'].astype(np.float32)
plt.bar(gender,Hgb)
plt.xlabel('Age')
plt.ylabel('Hgb')

In [ ]:
print('No. of samples = ', y.shape[0])

In [ ]:
x_seg = x_seg/255
y_seg = y_seg/20

In [ ]:
x_aug_it.shape

In [ ]:
X =np.zeros((1722,3,640,480))
X[:,0,:,:] = x_aug_it[:,:,:,0]
X[:,1,:,:] = x_aug_it[:,:,:,1]
X[:,2,:,:] = x_aug_it[:,:,:,2]
X.shape
plt.imshow(x_aug_it[0,:,:,:])

X[0,:,:,0]

## Training

In [ ]:
train_split=0.9
val_split = 0.05
test_split = 1 - train_split - val_split
batch_size = 3
epochs = 10

X=X[:,:,:]
X_in = torch.tensor(X).float()
X_in.size()

y_in = torch.tensor(y_seg).float().unsqueeze(1)

print(X_in.shape, y_in.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_use, X_test, y_use,  y_test = train_test_split(X_in, y_in, test_size=test_split)
X_train, X_val, y_train, y_val = train_test_split(X_use, y_use, test_size=val_split/(val_split+train_split))

In [ ]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
import torch.optim as optim
# criterion = nn.MSELoss
# #nn.BCEWithLogitsLoss()
# #nn.BCELoss()
# #nn.MSELoss()
# optimizer = optim.Adam(net.parameters(), lr=0.1)
# #optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(293904, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        # x = self.fc3(x)
        return x

net = Net()

In [ ]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
y_test 
# net(X_test)
# criterion(net(X_test), torch.Tensor(y_test))

criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

In [ ]:
batch_size = 5
epochs = 4

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    iters = (len(X_train)//batch_size)+1
    
    for i in range((len(X_train)//batch_size)+1):
        # get the inputs; data is a list of [inputs, labels]
        
        try:
            inputs = X_train[(i*batch_size):(i+1)*batch_size]
            labels = y_train[(i*batch_size):(i+1)*batch_size]
        except:
            inputs = X_train[(i*batch_size):]
            labels = y_train[(i*batch_size):]
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
#         print(outputs.shape, labels.shape, outputs.dtype, labels.dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
    with torch.no_grad():
        y_pred_val = net(X_val)
        val_loss = criterion(y_pred_val, torch.Tensor(y_val))
        validation_loss += val_loss.item()
        
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / iters:.9f}, val_loss: {val_loss.item():.3f}')
    # running_loss = 0.0
    # validation_loss = 0.0
    scheduler.step()

In [ ]:
torch.save(net.state_dict(), 'seg_vcnn_model_weights.pth')
net.load_state_dict(torch.load('seg_vcnn_model_weights.pth'))
net.eval()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#MSE
mean_squared_error( (net(X_test)).detach()*20 , y_test*20)

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(293904, 120)
        self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.25)
        self.bn = nn.BatchNorm2d(16)


    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.bn(x)
        x = self.pool(self.relu(self.conv2(x)))
        x = self.bn(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        # x = self.fc3(x)
        return x

net2 = Net2()

In [ ]:
criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

In [ ]:
from torchsummary import summary
summary(net2,(3,640,480))

In [ ]:
batch_size = 5
epochs = 4

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    iters = (len(X_train)//batch_size)+1
    
    for i in range((len(X_train)//batch_size)+1):
        # get the inputs; data is a list of [inputs, labels]
        
        try:
            inputs = X_train[(i*batch_size):(i+1)*batch_size]
            labels = y_train[(i*batch_size):(i+1)*batch_size]
        except:
            inputs = X_train[(i*batch_size):]
            labels = y_train[(i*batch_size):]
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
#         print(outputs.shape, labels.shape, outputs.dtype, labels.dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
    with torch.no_grad():
        y_pred_val = net2(X_val)
        # print(y_pred_val)
        val_loss = criterion(y_pred_val, torch.Tensor(y_val))
        validation_loss += val_loss.item()
        
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / iters:.9f}, val_loss: {val_loss.item():.3f}')
    # running_loss = 0.0
    # validation_loss = 0.0
    #scheduler.step()

In [ ]:
#MSE
mean_squared_error( (net2(X_test)).detach()*20 , y_test*20)

## Linear Regression

In [ ]:
# Import the packages and classes needed for this example:
from sklearn.linear_model import LinearRegression

'''y = np.load(r"y_base_italy.npy")
x = np.load(r"X_new_RGB.npy")/255'''

X = np.ones((211,307200))
for i in range(211):
  X[i] = x_seg[i,:,:,0].flatten()
X.shape,y_seg.shape

# # Create a linear regression model based the positioning of the data and Intercept, and predict a Best Fit:
model = LinearRegression(fit_intercept=True)
model.fit(X[:200], y_seg[:200])

In [ ]:
preds = model.predict(X[200:])
mean_squared_error( preds*20 , y_seg[200:]*20)

In [ ]:
X = np.ones((211,307200))
for i in range(211):
  X[i] = x_seg[i,:,:,1].flatten()
X.shape,y_seg.shape

# # Create a linear regression model based the positioning of the data and Intercept, and predict a Best Fit:
model = LinearRegression(fit_intercept=True)
model.fit(X[:200], y_seg[:200])

preds = model.predict(X[200:])
mean_squared_error( preds*20 , y_seg[200:]*20)

In [ ]:
X = np.ones((211,307200))
for i in range(211):
  X[i] = x_seg[i,:,:,2].flatten()
X.shape,y_seg.shape

# # Create a linear regression model based the positioning of the data and Intercept, and predict a Best Fit:
model = LinearRegression(fit_intercept=True)
model.fit(X[:200], y_seg[:200])

preds = model.predict(X[200:])
mean_squared_error( preds*20 , y_seg[200:]*20)

In [ ]:
X = np.ones((211,921600))
for i in range(211):
  X[i] = x_seg[i,:,:,:].flatten()
X.shape,y_seg.shape

# # Create a linear regression model based the positioning of the data and Intercept, and predict a Best Fit:
model = LinearRegression(fit_intercept=True)
model.fit(X[:200], y_seg[:200])

preds = model.predict(X[200:])
mean_squared_error( preds*20 , y_seg[200:]*20)

In [ ]:
y_seg[200:]

In [ ]:
preds 

## XGBoost

In [ ]:
import xgboost
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=10, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(X[:110], y_seg[:110])

In [ ]:

preds = model.predict(X[:20])
mean_squared_error( preds*20 , y_seg[:20]*20)

In [ ]:
preds